In [6]:
# Importing all needed functions
import os
import io
from contextlib import redirect_stdout
from astropy.io import fits
import glob
from Calculating_det_angles import estimate_source_angles_detectors  # Importing ma'am's function
from Tools import tools
import threading
from queue import Queue
import queue
import traceback
import pandas as pd
import os
from collections import Counter
import sys
import pandas as pd



In [7]:
# reading the ttsplit file
df = pd.read_csv('ttsplit')

# getting the test and train events
test_events = df[df['category'] == 'test']['filename'].tolist()

train_events = df[df['category'] == 'train']['filename'].tolist()

# loading the event names and event types in the list

event_names = []
event_types = []

for _,row in df.iterrows():
    if (row["category"] == "test" or row["category"] == "train") and row["filename"][-1] != 'd':
        event_type,event_name = row['filename'].split("_")
        event_types.append(event_type)
        event_names.append(event_name)

print(len(event_names))
print(len(event_types))


data_set_path = r"D:\GRB_data\new_dataset"

# Get a list of all folders in the specified directory
folders = [folder for folder in os.listdir(data_set_path) if os.path.isdir(os.path.join(data_set_path, folder))]

# Create sets for faster lookup
folder_set = set(folders)
event_types = set(['GRB', 'SFLARE', 'TGF', 'SGR'])

# Count events already downloaded
downloaded_events = Counter(folder.split('_')[0] for folder in folders if folder.split('_')[0] in event_types)
print("events already downloaded\n", dict(downloaded_events))

# Create set of existing event names for faster lookup
existing_event_names = set(folder.split('_')[1] for folder in folders)

# Initialize counters for events to download
events_to_download = Counter()

# Lists to store new events
new_event_types = []
new_event_names = []

event_limit = 100000
for _, row in df.iterrows():
    event_type,event_name = row['filename'].split("_")
    if (row["category"] == "test" or row["category"] == "train") and row["filename"][-1] != 'd':
        if event_type in event_types and event_name not in existing_event_names:
            if events_to_download[event_type] < event_limit - downloaded_events[event_type]:
                events_to_download[event_type] += 1
                new_event_types.append(event_type)
                new_event_names.append(event_name)

print('events to download\n', dict(events_to_download))

3492
3492
events already downloaded
 {'GRB': 1000, 'SFLARE': 1000, 'SGR': 492, 'TGF': 1000}
events to download
 {}


In [8]:

dir_path = tools.json_path(r'data_path.json')

# Function to process a single event
def process_event(event_name,thread_num):
    try:
        temp_thread = r'temp'+ str(thread_num)
        temp_path = os.path.join(dir_path, temp_thread)
        print(temp_path)
        tools.create_folder(temp_path)
        event = event_name
        year = '20' + event[2:4] + "/"

        # URL of the file you want to download : TRIGDAT file
        url = 'wget -q -nH --no-check-certificate --cut-dirs=7 -r -l0 -c -N -np -A "*_trigdat_*" -R "index*" -erobots=off --retr-symlinks https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/triggers/' + year + event + '/current/'
        tools.run_wget_download(url, temp_path)

        # Finding Trigdat file
        trig_string = "_trigdat_"
        trig_pattern = os.path.join(temp_path, 'current', f"*{trig_string}*")
        trigdat_file = glob.glob(trig_pattern)

        # Get the spacecraft pointing from here
        trigdat_filename = trigdat_file[0]

        # URL of the file you want to download : .rsp file
        url = 'wget -q -nH --no-check-certificate --cut-dirs=7 -r -l0 -c -N -np -A "*.rsp*" -R "index*" -erobots=off --retr-symlinks https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/triggers/' + year + event + '/current/'
        tools.run_wget_download(url, temp_path)

        # Finding .rsp file
        rsp_string = ".rsp"
        rsp_pattern = os.path.join(temp_path, 'current', f"*{trig_string}*")
        rsp_file = glob.glob(trig_pattern)

        # Getting the RA and DEC
        with fits.open(rsp_file[0], memmap=True) as pha_list:
            ra_obj, dec_obj = (pha_list[0].header['RA_OBJ']), (pha_list[0].header['DEC_OBJ'])

        brightest_nai, bright_nais, brightest_bgo,nai_angles = estimate_source_angles_detectors.angle_to_grb(ra_obj, dec_obj,trigdat_filename)

        # Get a list of all files in the folder
        files = [file for file in os.listdir(data_set_path) if not os.path.isdir(os.path.join(data_set_path, file))]

        # find the needed nais
        nais = []
        for nai_angle,nai in zip(nai_angles,bright_nais):
            if nai == brightest_nai or int(nai_angle) < 60: # only download the brightest detector or if the detector has anlge less than 60 deg
                nais.append(nai)

        nai_flags = []
        for nai in nais:
            flag = False
            for file in files:
                if nai in file:
                    flag = True
            nai_flags.append(flag)
        
        print(nai_flags)
            
        # Print the size of the queue
        print(event_queue.qsize()) 

    except Exception as e:
        # Get formatted traceback
        traceback_str = traceback.format_exc()

        # Access exception attributes (example)
        if isinstance(e, ZeroDivisionError):
            error_details = "Division by zero attempted."
        else:
            error_details = str(e.args[0]) if len(e.args) else "Unknown error."

        # Custom message based on exception type
        if isinstance(e, ValueError):
            custom_message = "Invalid value provided. Check your input data."
        else:
            custom_message = "An error occurred during processing."

        # Print detailed message
        print(f"""Error:
        - Event: {event_name}
        - Exception: {type(e).__name__}
        - Message: {e}
        - Details: {error_details}
        - Traceback:
        {traceback_str}
        - {custom_message}""")

# Create a queue to store the events
event_queue = Queue()

# Function to run the worker threads
def worker(thread_num):
    print(f"Worker thread {thread_num} started.")

    while True : #
        # Get an event from the queue
        event_name = event_queue.get()
        print(f"Worker thread {thread_num} processing event {event_type} {event_name}.")

        # Process the event
        process_event(event_name, thread_num)

        # Signal that the task is completed
        event_queue.task_done()
        print(f"Worker thread {thread_num} processed event {event_name}.")

        if event_queue.qsize() == 0:
            break
    
    print(f"Worker thread {thread_num} finished.")
    
# Create and start the worker threads
num_threads = 1
threads = []
for i in range(num_threads):
    worker_thread = threading.Thread(target=worker, args=(i,), daemon=True)
    worker_thread.start()
    threads.append(worker_thread)

# Add events to the queue
for folder in folders[:1]:
    event_queue.put((folder))

print(event_queue.qsize()) 

# # Wait for all events to be processed
# event_queue.join()
# print("All events have been processed.")

# Wait for all threads to finish
for thread in threads:
    thread.join()

print('done')

Worker thread 0 started.1

Worker thread 0 processing event SGR GRB_bn080714086.
C:\Users\arpan\OneDrive\Documents\GRB\data\temp0
Folder 'C:\Users\arpan\OneDrive\Documents\GRB\data\temp0' already exists.
Folder 'C:\Users\arpan\OneDrive\Documents\GRB\data\temp0' and its contents removed successfully.
Folder 'C:\Users\arpan\OneDrive\Documents\GRB\data\temp0' created successfully.
download start
Error downloading the file: Command 'wget -q -nH --no-check-certificate --cut-dirs=7 -r -l0 -c -N -np -A "*_trigdat_*" -R "index*" -erobots=off --retr-symlinks https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/triggers/20B_/GRB_bn080714086/current/ -P C:\Users\arpan\OneDrive\Documents\GRB\data\temp0' returned non-zero exit status 8.
Error:
        - Event: GRB_bn080714086
        - Exception: IndexError
        - Message: list index out of range
        - Details: list index out of range
        - Traceback:
        Traceback (most recent call last):
  File "C:\Users\arpan\AppData\Local\Temp\ipyker